# ChatGPT Prompts

Description todo

[Índice]

## Imports & Utils

In [4]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    """
    
    """
    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def save_text(path: str, text: str):
    """
    Helper function to save prompts and model responses as TXT files
    """
    
    with open(f"{path}.txt", "w") as text_file:
        text_file.write(text)

## Principle 1: Write clear and specific instructions

The best way to obtain the desired model output is to provide instructions that are as clear and specific as you can possibly make. This reduces the chances to get irrelevant or incorrect responses. However, don't confuse writing clear instructions with writing short instructions. In some cases, longer prompts actually provide more clarity and context information that will lead the model to generate more detailed and accurate responses.

Here are some tactics you can follow to get clear and specific prompts.

### 1. Use delimiters

Use delimiters in your prompt to separate the prompt instructions (system instructions) from the user input. You can use any punctuation symbol that clearly tell the model  the separation between the instructions and input.

Some delimiters examples:

Triple quotes `“““`

Triple back-ticks \`\`\`

Triple dashes `---`

Angle brackets `< >`

XML tags `<tag> </tag>`

Remember that long or verbose delimiters generates longer prompts (in tokens number), which can be a problem on some models with a short value of maximum prompt length.

Using delimiters is also a helpful technique to avoid prompting injections by explicitly separating the model instructions from the rest of the prompt (I will cover this in more detail later).

Let's see this tactic in practice with a real example. Imagine we are working to improve an e-commerce website providing better insights to the sales team based on customer reviews. For instance, if shoppers frequently mention that a product's sizing is either too large or too small, this could indicate that the actual size chart is inaccurate.

However, manually analyzing customer reviews is a tedious and time-consuming task. For this reason, we will use GPT-4 to examine these reviews and provide a comprehensive summary.

The following examples are actual reviews that online shoppers have given of a [Columbia Men's Republic Parka (Obi-Wan Kenobi™)](https://www.columbia.com/p/mens-republic-parka-obi-wan-kenobi-2049261_OWK.html). Which BTW looks soo cool!! If it didn't cost 500 bucks I would buy it too!

In [1]:
reviews = """
1 Star
This is an over engineered waste of money

Nobody is truly going to be wearing this unless your in the frozen tundra…. Zero mobility way to large in length… Absolute waste…

---
5 Stars
Incredibly warm!

Incredibly warm! You won’t go wrong with this jacket!

---
5 Stars
So warm!

I am a 5”4 woman and got this men’s jacket as a gift. I moved from sunny Los Ángeles to Detroit Michigan during the storm and this jacket saved me. I get cold easily and I was so warm while walking in the snow that I had to unzip a bit because I was getting too warm. Love that this jacket has so many pockets, I don’t even need to bring a purse cuz this jacket holds it all. The details they put into this jacket is amazing.

---
1 Star
Waste of money great look but just not designed to be used and it’s shown by the resale value

Absolutely way too bulky and does not fit as described… This jacket is over engineered meaning unless your going to the polar ice caps you won’t need it. Was hoping for a solid winter jacket and this is definitely not it. Zero mobility in this with how bulky it is… And I have other jackets by Columbia… Also Ordered an XL just as my other jackets but this one is basically a XXL the jacket is also extremely long for no reason. Definitely will be returning this as the package also came opened and the jacket was dirty.

---
2 Stars
Overpriced, and runs big

I love Star Wars and really loved this jacket in concept. It took forever to ship and when it did finally show up the package was ripped open and the jacket was dirty and damaged. Customer service was not helpful and I kept getting kicked from chat. They wanted me to return it and rebuy while doing so meanwhile I have spent $1000 and have no jacket for most of winter. Not worth the money or the hassle. Guess I am still a north face guy.

---
5 Stars
I love it!!

A very nice parka!! Comfy and warm and it looks great! Very glad I purchased it.

---
5 Stars
Awesome

Warmest jacket I've ever had. Lots of pockets, looks excellent

---
2 Stars
Very nice looking jacket

The jacket looks really nice and is warm . The sizing chart is completely off. I would normally be a large to extra large because of wide shoulders and back (I matched the large I'm the size chart). I can almost fit 2 of me into this jacket . Definitely go a size down , maybe even 2 . If you an athletic build .
"""

Let's start with a simple and specific prompt, indicating the model to analyze the customer's review and generate an overview of them. In the spirit of making more clean and easy-to-read examples, we will separate the system instructions from the rest of the input. To do this, we will use Python string literals.

In [2]:
prompt = f"""
You will be provided with a list of product reviews delimited by <>.
Analyze each reviews and provide an overview of customers satisfaction, listing 
positive and negative aspects reported by customers. 

<{reviews}>
"""

Our initial prompt is extremely concise and specific, with back-ticks to separate the instructions from the rest of the input. Despite its simplicity, the model gets the idea and generates a correct response, identifying and enumerating positives and negatives aspects mentioned by shoppers in their reviews. In particular, the model properly identifies highlighted aspects by shoppers like ample storage space, comfortable and warm feeling, among others. Analogous to positive aspects, recognize a few customers complains such as sizing issues and shipping issues.

Notice that the model cites some aspects that are not strictly included in the customer reviews. For instance: ample storage space. Customers mentioned multiple pockets but never said plainly "ample" (this is an implicit assumption). Also, none complain specifically about the jacket price. This discrepancy and inconsistencies are called Hallucinations, is a common issue in LLM. We will see later a tactic that helps to reduce it

Feel free to read through the customer's review and take your own insights, then compare them with the generated by the model.

In [6]:
response = get_completion(prompt)

In [8]:
print(response)

Overall, customers seem to have mixed feelings about this jacket. While some customers rave about how warm and comfortable it is, others feel that it is overpriced and bulky. The jacket seems to be well-designed with many pockets, but some customers feel that it is too large and lacks mobility. Additionally, some customers have had issues with shipping and customer service. It is recommended to carefully consider the sizing chart before purchasing.


### 2. Ask for structured output

Sometimes can be helpful to ask the model for structured outputs such as JSON, XML, or any other format supported by the model. Using this tactic reduces the complexity of the model response while making it easier to process the response using Python or any other coding language.  So let's go back to our initial prompt and update it asking the model to format the output in JSON format with two lists: positive and negative aspects.



### 3. Check whether conditions are satisfied

When designing application prompts based on user input, it is good practice to ensure that the input meets certain minimal criteria necessary to complete the instructions. If not, return a known error message. By doing so, we can guarantee that the model responds in a controlled manner to any input that exceeds the scope for which the prompt and the application were initially designed.

For instance, in the subsequent shopper review, another brand's jacket is being promoted. To prevent it from skewing our analysis, we will modify our prompt to automatically ignore this type of reviews.

### 4. Few-shot prompting

In some scenarios, providing examples of successful executions of the task and then asking the model to perform it is a smart strategy for more accurate and predictable results. To illustrate this tactic, lets momentarily think in a related functionality in e-commerce website. Imagine that now we want to automatically generate a response message for each customer review.

To do this, we are going to tell the model how to respond to some typical examples of product reviews.



# Principle 2: Give the model time to think

If you provide the model a problem which is to complex, he might need more time to think in a proper answer. This means spending more computational time to generate the response and also use more tokens in both input and output response.
You can think of this as asking the model to organize its reasoning process in a particular way, for example by following a series of steps, or performing certain tasks before generating a final answer.

Here are some tactics that can be used to instruct the model to think longer before generate a response.

### 1. Specify the steps to complete a task

The first tactic is to provide the model with a comprehensive list of steps to solve the task, the more specific the steps, the better. Let's go back to our shopper reviews and rewrite the model prompt to do both: get customer insights and respond to reviews.

### 2. Instruct the model to work out its own solution before rushing to a conclusion

The second tactic is similar to the previous one but in addition ask the model to first reach a solution by himself before rushing to a conclusion. This is similar to instead of instruct the model to say if something is correct or incorrect based on the context, instruct the model to first work on his own solution based on the context and then compare it with the input solution. For example if the task is validate students answers for a simple math problem like 2 + x = 4 then x=?, instruct the model to first find a solution and then compare it with the student’s solution.

To illustrate the power of this tactic, we will explore another feature of the e-commerce website: processing returns and refunds tickets. The idea is to give the model a list of actual shopper's refund demands, to process and evaluate if a refund applies based on the company policy. Here are six examples of refund requests for various reasons.